## Project 1: Clinical Trial Patient List (JOIN + CASE) 

## Using three data sets, identify patients that are eligible for a clinical trial based on the following criteria: 

    ** Group A ** 
    - Age: 18–30
    - Taking a weight-loss drug
    - Diagnosed with diabetes (assume ICD-10 code “E11”)
    - Admitted in the last 24 months

    ** Group B **
    - Age: Over 30
    - Taking a weight-loss drug
    - Diagnosed with hypertension (assume ICD-10 code “I10”)
    - Admitted in the last 24 months
---

## Table Structures

1. `patient_vitals`

    * Patient_ID
    * Age, Gender, Blood_Type, Height, Weight, BMI, etc.

2. `patient_rx` (medication info)

    * Patient_ID
    * Weight_Loss_Drug, Hypertension_Drug, Diabetes_Drug, etc. (e.g., 'Y' or 'N')

3. `encounters`

    * Patient_ID
    * Encounter_ID, Admission_Date, Discharge_Date, Diagnosis_Code, etc.

## Tech Stack
- **Database:** PostgreSQL
- **Language:** SQL

### Analytical Approach
- Temporal validation: To ensure that the weight loss drug was prescribed on or before the encounter date.
- 





---
## Approach: SQL Query (JOIN, CASE)

1. All three tables are joined using PATIENT_ID as the common field
2. CASE block is used to check against the given inclusion criteria
4. The patient is sorted into trial_group Group A or Group B if the criteria is 'Not eligible' and excluded from the output.
---

In [ ]:
-- Common Table Expressions (CTEs) for better readability and maintenance
WITH 
-- 1. Filter patients by recent diagnoses
diagnosis_filter AS (
    SELECT
        e."Encounter_ID",
        e."Patient_ID",
        e."Admission_Date",
        e."Diagnosis_Code"
    FROM "encounters" e
    WHERE e."Admission_Date" >= CURRENT_DATE - INTERVAL '24 months'
      AND e."Diagnosis_Code" IN ('E11', 'I10')
),

-- 2. Get eligible patients with medications
medication_eligibility AS (
    SELECT DISTINCT
        v."Patient_ID",
        v."Age",
        rx."Weight_Loss_Drug",
        rx."Prescribed_Date"
    FROM "patient_vitals" v
    INNER JOIN Patient_RX rx 
        ON v."Patient_ID" = rx."Patient_ID"
    WHERE rx."Weight_Loss_Drug" IS NOT NULL
),

-- 3. Assign trial groups based on criteria
trial_assignment AS (
    SELECT
        d."Patient_ID",
        d."Encounter_ID",
        d."Admission_Date",
        d."Diagnosis_Code",
        m."Age",
        m."Weight_Loss_Drug",
        CASE
            WHEN m."Age" BETWEEN 18 AND 30 
             AND d."Diagnosis_Code" = 'E11' THEN 'Group A'
            WHEN m."Age" > 30 
             AND d."Diagnosis_Code" = 'I10' THEN 'Group B'
            ELSE 'Not eligible'
        END AS trial_group
    FROM diagnosis_filter d
    INNER JOIN medication_eligibility m 
        ON d."Patient_ID" = m."Patient_ID"
        AND m."Prescribed_Date" <= d."Admission_Date"
)

-- Final selection with clear filtering
SELECT DISTINCT
    "Patient_ID",
    "Encounter_ID",
    "Admission_Date",
    "Diagnosis_Code",
    "Age",
    "Weight_Loss_Drug",
    trial_group
FROM trial_assignment
WHERE trial_group IN ('Group A', 'Group B')
ORDER BY trial_group;